In [81]:
using Pkg
Pkg.activate("CT")
using CT

  Activating project at `~/CT_MPS_mini/CT`


In [82]:
Pkg.status()

Project CT v0.1.0
Status `~/CT_MPS_mini/CT/Project.toml`
  [c7e460c6] ArgParse v1.2.0
⌃ [6e4b80f9] BenchmarkTools v1.5.0
⌅ [9136182c] ITensors v0.6.16
  [682c06a0] JSON v0.21.4
⌃ [33e6dc65] MKL v0.7.0
⌃ [91a5bcdd] Plots v1.40.5
⌃ [9f0aa9f4] TCIITensorConversion v0.1.4
⌃ [b261b2ec] TensorCrossInterpolation v0.9.10
⌃ [a759f4b9] TimerOutputs v0.5.24
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [83]:
# benchmark adder_MPO
using ITensors
include("CT/src/test_adder_MPO.jl")
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO
using Random
L = 4
ancilla = 0
folded = true
seed_vec = 123457
xj = Set([1//3, 2//3])
i1 = 1
p_ctrl = 1.0
p_proj = 0.0
_maxdim = 50
_cutoff = 1e-15
seed = 123457
x0 = nothing
qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
println(phy_list)
println(ram_phy)
println(phy_ram)
rng = MersenneTwister(seed_vec)
rng_vec = seed_vec === nothing ? rng : MersenneTwister(seed_vec)
# initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim);
# println(initial_state)
initial_state = productMPS(qubit_site, [1,2,2,1])

[1, 2, 3, 4]
[1, 4, 2, 3]
[1, 3, 4, 2]


MPS
[1] ((dim=2|id=742|"Qubit,Site,n=1"), (dim=1|id=395|"Link,l=1"))
[2] ((dim=1|id=395|"Link,l=1"), (dim=2|id=966|"Qubit,Site,n=2"), (dim=1|id=753|"Link,l=2"))
[3] ((dim=1|id=753|"Link,l=2"), (dim=2|id=586|"Qubit,Site,n=3"), (dim=1|id=421|"Link,l=3"))
[4] ((dim=1|id=421|"Link,l=3"), (dim=2|id=111|"Qubit,Site,n=4"))


In [87]:
shift_1_3_bits, shift_1_3_amount = fraction_to_binary_shift(1, 3, L)
L = length(shift_1_3_bits)
    
# Create all unique carry indices first
all_carry_labels = unique(vcat(phy_list, phy_list .+ 1))
# println(all_carry_labels)
carry_indices = Dict(label => Index(2, "Carry, c=$label") for label in all_carry_labels)
# println(carry_indices)    
# Now use the same index objects
dict = Dict()
for i in 1:L
    dict[phy_list[i]] = Dict(
        "in" => carry_indices[phy_list[i] + 1],
        "out" => carry_indices[phy_list[i]]
    )
end
# println(dict)

# if folded == true, then attach tensor T where T[1,:,:,1] = Identity and T[2,:,:,2] = Identity to the existing T[c_out, s, s_prime, c_in]

function create_identity_tensor_4d(c_out::Index, s::Index, s_prime::Index, c_in::Index)
    id_op = ITensor(c_out, s, s_prime, c_in)
    for outer in 1:2  # first and last index values
        for inner in 1:2  # diagonal of the 2×2 identity
            id_op[c_out => outer, s => inner, s_prime => inner, c_in => outer] = 1.0
        end
    end
    return id_op
end

lsb = 4
ram_pos = phy_ram[lsb]
shift_bits = [0,1,0,1]
shift_bit = shift_bits[ram_phy[ram_pos]]
println(shift_bit, ram_pos)
T = create_addition_tensor_with_carry(shift_bit, qubit_site[ram_pos], prime(qubit_site[ram_pos]), dict[ram_pos]["in"], dict[ram_pos]["out"]);

id_op = create_identity_tensor_4d(qubit_site[ram_pos+1], prime(qubit_site[ram_pos+1]), dict[ram_pos+1]["in"], dict[ram_pos+1]["out"]);

# println(inds(T))
# println(inds(id_op))

# println(filterinds(T, tags="Site"))
# # attach id to T
T_new = T * id_op;
# println(inds(T_new)[1])
# println(inds(T_new))
# println(inds(initial_state))
# # contract the first two qubit sites 
initial_state_new = contract(initial_state[ram_pos], initial_state[ram_pos+1])

# println(inds(initial_state_new))
# println(inds(T_new))

# println(typeof(initial_state_new))
# initial_state_new = MPS(initial_state_new, filterinds(initial_state_new, tags="Site"))
# println(inds(initial_state_new))
println(typeof(T_new), inds(T_new))
println(typeof(initial_state_new), inds(initial_state_new))
result = contract(T_new, initial_state_new)
println(inds(result))
# println(result)

12
ITensor((dim=2|id=966|"Qubit,Site,n=2"), (dim=2|id=966|"Qubit,Site,n=2")', (dim=2|id=753|"Carry,c=2"), (dim=2|id=586|"Qubit,Site,n=3"), (dim=2|id=586|"Qubit,Site,n=3")', (dim=2|id=782|"Carry,c=4"))
ITensor((dim=1|id=395|"Link,l=1"), (dim=2|id=966|"Qubit,Site,n=2"), (dim=2|id=586|"Qubit,Site,n=3"), (dim=1|id=421|"Link,l=3"))
((dim=2|id=966|"Qubit,Site,n=2")', (dim=2|id=753|"Carry,c=2"), (dim=2|id=586|"Qubit,Site,n=3")', (dim=2|id=782|"Carry,c=4"), (dim=1|id=395|"Link,l=1"), (dim=1|id=421|"Link,l=3"))


In [85]:
T_new * initial_state[1]

ITensor ord=8 (dim=2|id=966|"Qubit,Site,n=2") (dim=2|id=966|"Qubit,Site,n=2")' (dim=2|id=271|"Carry,c=2") (dim=2|id=586|"Qubit,Site,n=3") (dim=2|id=586|"Qubit,Site,n=3")' (dim=2|id=633|"Carry,c=4") (dim=2|id=742|"Qubit,Site,n=1") (dim=1|id=395|"Link,l=1")
NDTensors.Dense{Float64, Vector{Float64}}

In [86]:
function print_mpo_as_matrix(mpo, L)
    println("\nMPO as matrix:")
    
    # Contract the entire MPO to get the matrix representation
    mpo_tensor = contract(mpo)
    
    # The tensor has indices (s1', s2', ..., sL', s1, s2, ..., sL)
    # We need to reshape it into a matrix where the first L indices are rows
    # and the last L indices are columns
    
    # Get the tensor as an array
    tensor_array = Array(mpo_tensor, inds(mpo_tensor)...)
    
    # Reshape to matrix: first 2^L entries are rows, next 2^L are columns
    matrix = reshape(tensor_array, 2^L, 2^L)
    
    # Create basis labels
    basis_labels = []
    for i in 0:2^L-1
        bits = string(i, base=2, pad=L)
        push!(basis_labels, "|$bits⟩")
    end
    
    # Print header
    print("      ")
    for j in 1:2^L
        print("$(basis_labels[j])  ")
    end
    println()
    
    # Print matrix rows
    for i in 1:2^L
        print("$(basis_labels[i]) ")
        for j in 1:2^L
            val = abs(matrix[i, j])
            if val > 0.5
                print("  1   ")
            else
                print("  0   ")
            end
        end
        println()
    end
    
    return matrix
end

print_mpo_as_matrix(mpo, 4)

LoadError: UndefVarError: `mpo` not defined

In [ ]:
function print_mpo_as_matrix(mpo, L)
    println("\nMPO as matrix:")
    
    # Contract the entire MPO to get the matrix representation
    mpo_tensor = contract(mpo)
    
    # The tensor has indices (s1', s2', ..., sL', s1, s2, ..., sL)
    # We need to reshape it into a matrix where the first L indices are rows
    # and the last L indices are columns
    
    # Get the tensor as an array
    tensor_array = Array(mpo_tensor, inds(mpo_tensor)...)
    
    # Reshape to matrix: first 2^L entries are rows, next 2^L are columns
    matrix = reshape(tensor_array, 2^L, 2^L)
    
    # Create basis labels
    basis_labels = []
    for i in 0:2^L-1
        bits = string(i, base=2, pad=L)
        push!(basis_labels, "|$bits⟩")
    end
    
    # Print header
    print("      ")
    for j in 1:2^L
        print("$(basis_labels[j])  ")
    end
    println()
    
    # Print matrix rows
    for i in 1:2^L
        print("$(basis_labels[i]) ")
        for j in 1:2^L
            val = abs(matrix[i, j])
            if val > 0.5
                print("  1   ")
            else
                print("  0   ")
            end
        end
        println()
    end
    
    return matrix
end

print_mpo_as_matrix(mpo, 4)


MPO as matrix:
      |0000⟩  |0001⟩  |0010⟩  |0011⟩  |0100⟩  |0101⟩  |0110⟩  |0111⟩  |1000⟩  |1001⟩  |1010⟩  |1011⟩  |1100⟩  |1101⟩  |1110⟩  |1111⟩  
|0000⟩   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0   
|0001⟩   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0   
|0010⟩   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0   
|0011⟩   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0   
|0100⟩   0     1     0     0     0     0     0     0     0     0     0     0     0     1     0     0   
|0101⟩   0     0     1     0     0     0     0     0     0     0     0     0     0     0     1     0   
|0110⟩   0     0     1     0     0     0     0     0     0     0     0     0     0     0     1     0   
|0111⟩   0     1     0     0     0     0     0     0     0     0     0     0     0     1     0     0   
|1000⟩   0     0 

16×16 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  1.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0     1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  …  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.

In [ ]:
function print_mpo_as_matrix(mpo, L)
    println("\nMPO as matrix:")
    
    # Contract the entire MPO to get the matrix representation
    mpo_tensor = contract(mpo)
    
    # The tensor has indices (s1', s2', ..., sL', s1, s2, ..., sL)
    # We need to reshape it into a matrix where the first L indices are rows
    # and the last L indices are columns
    
    # Get the tensor as an array
    tensor_array = Array(mpo_tensor, inds(mpo_tensor)...)
    
    # Reshape to matrix: first 2^L entries are rows, next 2^L are columns
    matrix = reshape(tensor_array, 2^L, 2^L)
    
    # Create basis labels
    basis_labels = []
    for i in 0:2^L-1
        bits = string(i, base=2, pad=L)
        push!(basis_labels, "|$bits⟩")
    end
    
    # Print header
    print("      ")
    for j in 1:2^L
        print("$(basis_labels[j])  ")
    end
    println()
    
    # Print matrix rows
    for i in 1:2^L
        print("$(basis_labels[i]) ")
        for j in 1:2^L
            val = abs(matrix[i, j])
            if val > 0.5
                print("  1   ")
            else
                print("  0   ")
            end
        end
        println()
    end
    
    return matrix
end

print_mpo_as_matrix(mpo, 4)


MPO as matrix:
      |0000⟩  |0001⟩  |0010⟩  |0011⟩  |0100⟩  |0101⟩  |0110⟩  |0111⟩  |1000⟩  |1001⟩  |1010⟩  |1011⟩  |1100⟩  |1101⟩  |1110⟩  |1111⟩  
|0000⟩   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0   
|0001⟩   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0   
|0010⟩   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0   
|0011⟩   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0   
|0100⟩   0     1     0     0     0     0     0     0     0     0     0     0     0     1     0     0   
|0101⟩   0     0     1     0     0     0     0     0     0     0     0     0     0     0     1     0   
|0110⟩   0     0     1     0     0     0     0     0     0     0     0     0     0     0     1     0   
|0111⟩   0     1     0     0     0     0     0     0     0     0     0     0     0     1     0     0   
|1000⟩   0     0 

16×16 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  1.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0     1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  …  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.

In [ ]:
function print_mpo_as_matrix(mpo, L)
    println("\nMPO as matrix:")
    
    # Contract the entire MPO to get the matrix representation
    mpo_tensor = contract(mpo)
    
    # The tensor has indices (s1', s2', ..., sL', s1, s2, ..., sL)
    # We need to reshape it into a matrix where the first L indices are rows
    # and the last L indices are columns
    
    # Get the tensor as an array
    tensor_array = Array(mpo_tensor, inds(mpo_tensor)...)
    
    # Reshape to matrix: first 2^L entries are rows, next 2^L are columns
    matrix = reshape(tensor_array, 2^L, 2^L)
    
    # Create basis labels
    basis_labels = []
    for i in 0:2^L-1
        bits = string(i, base=2, pad=L)
        push!(basis_labels, "|$bits⟩")
    end
    
    # Print header
    print("      ")
    for j in 1:2^L
        print("$(basis_labels[j])  ")
    end
    println()
    
    # Print matrix rows
    for i in 1:2^L
        print("$(basis_labels[i]) ")
        for j in 1:2^L
            val = abs(matrix[i, j])
            if val > 0.5
                print("  1   ")
            else
                print("  0   ")
            end
        end
        println()
    end
    
    return matrix
end

print_mpo_as_matrix(mpo, 4)


MPO as matrix:
      |0000⟩  |0001⟩  |0010⟩  |0011⟩  |0100⟩  |0101⟩  |0110⟩  |0111⟩  |1000⟩  |1001⟩  |1010⟩  |1011⟩  |1100⟩  |1101⟩  |1110⟩  |1111⟩  
|0000⟩   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0   
|0001⟩   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0   
|0010⟩   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0   
|0011⟩   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0   
|0100⟩   0     1     0     0     0     0     0     0     0     0     0     0     0     1     0     0   
|0101⟩   0     0     1     0     0     0     0     0     0     0     0     0     0     0     1     0   
|0110⟩   0     0     1     0     0     0     0     0     0     0     0     0     0     0     1     0   
|0111⟩   0     1     0     0     0     0     0     0     0     0     0     0     0     1     0     0   
|1000⟩   0     0 

16×16 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  1.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0     1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  …  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.

In [ ]:
# Create 2D identity matrices to compare against
expected_identity = [1.0 0.0; 0.0 1.0]

# Extract T[1,:,:,1] as a matrix
slice_1 = zeros(2, 2)
for i in 1:2, j in 1:2
    slice_1[i, j] = T[i1 => 1, i2 => i, i3 => j, i4 => 1]
end

# Extract T[2,:,:,2] as a matrix  
slice_2 = zeros(2, 2)
for i in 1:2, j in 1:2
    slice_2[i, j] = T[i1 => 2, i2 => i, i3 => j, i4 => 2]
end

println("T[1,:,:,1] =")
display(slice_1)
println("Expected identity =")
display(expected_identity)
println("Are they equal? ", slice_1 ≈ expected_identity)

println("\nT[2,:,:,2] =")
display(slice_2)
println("Are they equal? ", slice_2 ≈ expected_identity)

In [ ]:
function print_mpo_as_matrix(mpo, L)
    println("\nMPO as matrix:")
    
    # Contract the entire MPO to get the matrix representation
    mpo_tensor = contract(mpo)
    
    # The tensor has indices (s1', s2', ..., sL', s1, s2, ..., sL)
    # We need to reshape it into a matrix where the first L indices are rows
    # and the last L indices are columns
    
    # Get the tensor as an array
    tensor_array = Array(mpo_tensor, inds(mpo_tensor)...)
    
    # Reshape to matrix: first 2^L entries are rows, next 2^L are columns
    matrix = reshape(tensor_array, 2^L, 2^L)
    
    # Create basis labels
    basis_labels = []
    for i in 0:2^L-1
        bits = string(i, base=2, pad=L)
        push!(basis_labels, "|$bits⟩")
    end
    
    # Print header
    print("      ")
    for j in 1:2^L
        print("$(basis_labels[j])  ")
    end
    println()
    
    # Print matrix rows
    for i in 1:2^L
        print("$(basis_labels[i]) ")
        for j in 1:2^L
            val = abs(matrix[i, j])
            if val > 0.5
                print("  1   ")
            else
                print("  0   ")
            end
        end
        println()
    end
    
    return matrix
end

print_mpo_as_matrix(mpo, 4)


MPO as matrix:
      |0000⟩  |0001⟩  |0010⟩  |0011⟩  |0100⟩  |0101⟩  |0110⟩  |0111⟩  |1000⟩  |1001⟩  |1010⟩  |1011⟩  |1100⟩  |1101⟩  |1110⟩  |1111⟩  
|0000⟩   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0   
|0001⟩   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0   
|0010⟩   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0   
|0011⟩   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0   
|0100⟩   0     1     0     0     0     0     0     0     0     0     0     0     0     1     0     0   
|0101⟩   0     0     1     0     0     0     0     0     0     0     0     0     0     0     1     0   
|0110⟩   0     0     1     0     0     0     0     0     0     0     0     0     0     0     1     0   
|0111⟩   0     1     0     0     0     0     0     0     0     0     0     0     0     1     0     0   
|1000⟩   0     0 

16×16 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  1.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0     1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0  …  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.

In [ ]:
# debug 
# haining
using .CT: add1, power_mpo
add1_mpo=MPO(add1(i1,L,phy_ram,phy_list),qubit_site)
add1_6,add1_3=power_mpo(add1_mpo,[div(2^L,6)+1,div(2^L,3)])

# tao
# mpo, bond_dims, dense_mat, A_tensor = mpo_adder_1_3(L, qubit_site, ram_phy, phy_ram, phy_list, folded);
shift_1_3_bits, shift_1_3_amount = fraction_to_binary_shift(1, 3, L)
mpo = create_binary_addition_mpo(shift_1_3_bits, qubit_site, phy_ram, phy_list, ram_phy)

result_h = initial_state * add1_3
result_t = initial_state * mpo
n = 3
println(result_h[n])
println(result_t[n])

ITensor ord=3
Dim 1: (dim=2|id=465|"Qubit,Site,n=3")'
Dim 2: (dim=2|id=327|"Link,l=3")'
Dim 3: (dim=4|id=264|"Link,l=2")'
NDTensors.Dense{Float64, Vector{Float64}}
 2×2×4
[:, :, 1] =
 -0.7609666965605797  -0.03853975873556803
 -0.6459528158451244   0.04679031335691027

[:, :, 2] =
 -0.008436923289067422   0.9892071090545564
 -0.05878317408817679   -0.13395018538792797

[:, :, 3] =
  0.6195590000078276  -0.07169420905829468
 -0.7431507543690252  -0.24235004059738682

[:, :, 4] =
  0.19236722839505166  0.1218356383823573
 -0.16436682885127002  0.9597575069948381
ITensor ord=3
Dim 1: (dim=2|id=465|"Qubit,Site,n=3")'
Dim 2: (dim=2|id=977|"Link,l=3")'
Dim 3: (dim=4|id=984|"Link,l=2")'
NDTensors.Dense{Float64, Vector{Float64}}
 2×2×4
[:, :, 1] =
 -0.7609666965605794  -0.03853975873556803
 -0.6459528158451244   0.04679031335691022

[:, :, 2] =
 -0.008436923289067533   0.9892071090545561
 -0.05878317408817668   -0.13395018538792788

[:, :, 3] =
  0.6195590000078279  -0.07169420905829474
 -0.74

In [ ]:
# L = 6
# folded = true
# qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
# mpo, bond_dims, dense_mat, A_tensor = mpo_adder_1_3(L, qubit_site, ram_phy, phy_ram, phy_list, folded);
# for n in 1:L
#     # println(n)
#     display(mpo_matrix_form(mpo, n))
# end